In [1]:
print('Hello world')

Hello world


In [2]:
from neo4j import GraphDatabase

In [3]:
uri = "bolt://localhost:7690"  # Change if needed
user = "neo4j"
password = "odi50graph"

In [4]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [5]:
query = """
MATCH (a)-[r]->(b)
RETURN a, r, b
"""

In [6]:
with driver.session() as session:
    session.run(query)
    print("Query executed successfully.")

Query executed successfully.


In [21]:
with driver.session() as session:
    result = session.run(query)
    for record in result:
        node = record["n"]
        print(dict(node))

KeyError: 'n'

In [7]:
with driver.session() as session:
    result = session.run(query)
    for record in result:
        a = record["a"]
        b = record["b"]
        r = record["r"]
        
        print(f"({list(a.labels)}: {dict(a)}) -[:{r.type}]-> ({list(b.labels)}: {dict(b)})")

(['Over']: {'number': 21, 'bowler': 'Kuldeep Yadav'}) -[:HAS_DELIVERY]-> (['Delivery']: {'runs_total': 1, 'batter': 'M Labuschagne', 'non_striker': 'TM Head', 'runs_extras': 0, 'runs_batter': 1})
(['Over']: {'number': 21, 'bowler': 'Kuldeep Yadav'}) -[:HAS_DELIVERY]-> (['Delivery']: {'runs_total': 4, 'batter': 'M Labuschagne', 'non_striker': 'TM Head', 'runs_extras': 0, 'runs_batter': 4})
(['Over']: {'number': 21, 'bowler': 'Kuldeep Yadav'}) -[:HAS_DELIVERY]-> (['Delivery']: {'runs_total': 1, 'batter': 'TM Head', 'non_striker': 'M Labuschagne', 'runs_extras': 0, 'runs_batter': 1})
(['Over']: {'number': 21, 'bowler': 'Kuldeep Yadav'}) -[:HAS_DELIVERY]-> (['Delivery']: {'runs_total': 0, 'batter': 'TM Head', 'non_striker': 'M Labuschagne', 'runs_extras': 0, 'runs_batter': 0})
(['Over']: {'number': 21, 'bowler': 'Kuldeep Yadav'}) -[:HAS_DELIVERY]-> (['Delivery']: {'runs_total': 0, 'batter': 'M Labuschagne', 'non_striker': 'TM Head', 'runs_extras': 0, 'runs_batter': 0})
(['Over']: {'number'

In [ ]:
from langchain_community.llms import HuggingFaceHub
import os
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = 'token'

llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-v0.1", model_kwargs={"temperature": 0.7})


C:\Users\Pratik\AppData\Local\Temp\ipykernel_7376\3404621281.py:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-v0.1", model_kwargs={"temperature": 0.7})
c:\Users\Pratik\Desktop\R&D\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="path/to/mistral-7b-instruct.gguf",
    temperature=0.7,
    max_tokens=256
)

In [9]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph

In [10]:
graph = Neo4jGraph(
    url="bolt://localhost:7690",
    username="neo4j",
    password="odi50graph" 
)

C:\Users\Pratik\AppData\Local\Temp\ipykernel_21760\1947756141.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [10]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# How many boundaries (fours and sixes) did TM Head hit?
  MATCH (d:Delivery)
  WHERE d.batter = "TM Head" AND (d.runs_batter = 4 OR d.runs_batter = 6)
  RETURN d;   

# What was the method of dismissal for V Kohli?
  MATCH (d:Delivery)-[:HAS_DISMISSAL]->(w:Dismissal)
  WHERE w.player_out = "V Kohli"
  RETURN w.kind AS DismissalMethod;

  
# How many wickets did JJ Bumrah take?
  MATCH (o:Over)-[:HAS_DELIVERY]-> (d:Delivery)-[:HAS_DISMISSAL]->(w:Dismissal)
  WHERE o.bowler = "JJ Bumrah"
  RETURN  COUNT(DISTINCT w) AS Wickets;

The question is:
{question}"""

In [11]:
from langchain.prompts.prompt import PromptTemplate


In [14]:
# Check the graph schema
import textwrap
graph.refresh_schema()
print(textwrap.fill(graph.schema,60))

Node properties: Event {name: STRING} Match {stage: STRING,
city: STRING, date: STRING, number: INTEGER, type: STRING,
venue: STRING} Team {name: STRING} Player {name: STRING}
Outcome {by_wickets: INTEGER, winner: STRING,
by_balls_remaining: INTEGER} Innings {team: STRING} Over
{number: INTEGER, bowler: STRING} Delivery {batter: STRING,
non_striker: STRING, runs_batter: INTEGER, runs_total:
INTEGER, runs_extras: INTEGER} Dismissal {kind: STRING,
player_out: STRING} Relationship properties:  The
relationships: (:Event)-[:HAS_MATCH]->(:Match)
(:Match)-[:HAS_OUTCOME]->(:Outcome)
(:Match)-[:HAS_INNINGS]->(:Innings)
(:Team)-[:PARTICIPATED_IN]->(:Match)
(:Team)-[:CONTAINS]->(:Player)
(:Innings)-[:HAS_OVERS]->(:Over)
(:Over)-[:HAS_DELIVERY]->(:Delivery)
(:Delivery)-[:HAS_DISMISSAL]->(:Dismissal)


In [13]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [14]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral")

C:\Users\Pratik\AppData\Local\Temp\ipykernel_7376\3305178565.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


In [15]:
chain = GraphCypherQAChain.from_llm(llm, graph=graph, verbose=True,
        allow_dangerous_requests=True,
        cypher_prompt=CYPHER_GENERATION_PROMPT)

In [44]:
response = chain.run("What was the method of dismissal for TM Head?")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (d:Delivery)-[:HAS_DISMISSAL]->(w:Dismissal)
    WHERE d.batter = "TM Head"
    RETURN w.kind AS DismissalMethod;
Full Context:
[{'DismissalMethod': 'caught'}]

> Finished chain.
 The method of dismissal for the TM Head was 'caught'.


In [ ]:
response = chain.run("List all the players of team India.")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (t:Team {name: "India"})-[:CONTAINS]->(p:Player)
RETURN p;
Full Context:
[{'p': {'name': 'RG Sharma'}}, {'p': {'name': 'Shubman Gill'}}, {'p': {'name': 'V Kohli'}}, {'p': {'name': 'SS Iyer'}}, {'p': {'name': 'KL Rahul'}}, {'p': {'name': 'RA Jadeja'}}, {'p': {'name': 'SA Yadav'}}, {'p': {'name': 'Mohammed Shami'}}, {'p': {'name': 'JJ Bumrah'}}, {'p': {'name': 'Kuldeep Yadav'}}]

> Finished chain.
 RG Sharma, Shubman Gill, V Kohli, SS Iyer, KL Rahul, RA Jadeja, SA Yadav, Mohammed Shami, JJ Bumrah, Kuldeep Yadav are the players of Team India.


In [41]:
response = chain.run("Who was the player of the match?")
print(response)



> Entering new GraphCypherQAChain chain...


OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama3.3`.

In [21]:
response = chain.run("List all the players of team India?")
print(response)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (t:Team {name:'India'})-[:CONTAINS]->(p:Player)
    RETURN p.name
Full Context:
[{'p.name': 'RG Sharma'}, {'p.name': 'Shubman Gill'}, {'p.name': 'V Kohli'}, {'p.name': 'SS Iyer'}, {'p.name': 'KL Rahul'}, {'p.name': 'RA Jadeja'}, {'p.name': 'SA Yadav'}, {'p.name': 'Mohammed Shami'}, {'p.name': 'JJ Bumrah'}, {'p.name': 'Kuldeep Yadav'}]

> Finished chain.
 RG Sharma, Shubman Gill, V Kohli, SS Iyer, KL Rahul, RA Jadeja, SA Yadav, Mohammed Shami, JJ Bumrah, Kuldeep Yadav are the players of Team India.


In [16]:
def qa_chain(query):
    response = chain.run(query)
    return response

In [17]:
import gradio as gr

iface = gr.Interface(fn=qa_chain, 
                     inputs=gr.Textbox(lines=2, placeholder="Ask a question..."), flagging_mode= "never",
                     outputs="text")

iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\Pratik\AppData\Local\Temp\ipykernel_7376\1133015883.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(query)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (t:Team {name:'India'})-[:CONTAINS]->(p:Player)
   RETURN p;
Full Context:
[{'p': {'name': 'RG Sharma'}}, {'p': {'name': 'Shubman Gill'}}, {'p': {'name': 'V Kohli'}}, {'p': {'name': 'SS Iyer'}}, {'p': {'name': 'KL Rahul'}}, {'p': {'name': 'RA Jadeja'}}, {'p': {'name': 'SA Yadav'}}, {'p': {'name': 'Mohammed Shami'}}, {'p': {'name': 'JJ Bumrah'}}, {'p': {'name': 'Kuldeep Yadav'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (m:Match)-[:HAS_OUTCOME]->(o:Outcome)
   RETURN o.winner AS GameWinner;
Full Context:
[{'GameWinner': 'Australia'}, {'GameWinner': 'Australia'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (t:Team)-[:PARTICIPATED_IN]->(m:Match {type: 'World Cup'})-[:HAS_OUTCOME]->(o:Outcome)<-[:CONTAINS]-(captain:Player)
  WHERE t.name = "India" AND count(distinct m) >= 2
  RETURN captain;


Traceback (most recent call last):
  File "c:\Users\Pratik\Desktop\R&D\venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\Pratik\Desktop\R&D\venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\Pratik\Desktop\R&D\venv\Lib\site-packages\gradio\blocks.py", line 2136, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\Pratik\Desktop\R&D\venv\Lib\site-packages\gradio\blocks.py", line 1662, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        fn, *processed_input,